<a href="https://colab.research.google.com/github/Vicks-2019/NLP/blob/master/Sentiment_Analysis_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Sentiment analysis using RNN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Review.xlsx to Review.xlsx


In [3]:
df=pd.read_excel('Review.xlsx')
df.head(4)

,Review,Lable
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1


In [4]:
df.columns

Index(['Review  ', 'Lable'], dtype='object')

In [8]:

x=df['Review  ']
y=df['Lable']

In [9]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [10]:
print("shape of X_train: ",X_train.shape)
print("shape of X_test: ",X_test.shape)

print("shape of y_train: ",y_train.shape)
print("shape of y_test: ",y_test.shape)

shape of X_train:  (39,)
shape of X_test:  (10,)
shape of y_train:  (39,)
shape of y_test:  (10,)


In [11]:

print(X_train[6]) 
print(y_train[6])

Honeslty it didn't taste THAT fresh.)   
0


In [12]:

# we will use the Tokenizer class : create a word-to-index dictionary. In the word-to-index dictionary,
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000,lower=True)
tokenizer.fit_on_texts(X_train)



X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

In [13]:

print(X_train[6]) 
print(y_train[6])

[4, 2, 75, 37, 38, 76, 77, 78, 23]
0


In [14]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100  #We set the maximum size of each list to 100.


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[3, :])  #the resulting feature vector contains mostly zeros, since you have a fairly short sentence.

[66  6  8 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


In [15]:
from keras.utils import to_categorical
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])
#print(len(y_train[0]))

(39, 2)
[1. 0.]


In [16]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
from keras.preprocessing.sequence import pad_sequences
import numpy as np
#import numpy as np

#(samples, timesteps, features)
#(batch_size, timesteps, input_dim)
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)  #(750, 100, 1)

X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

(39, 100, 1)


In [17]:

num_classes=2

def vanilla_rnn():
    model = Sequential()
    #model.add(SimpleRNN(50, input_shape = (X_train.shape[1], X_train.shape[2],), return_sequences = False))
    model.add(SimpleRNN(50, input_shape = (maxlen,1), return_sequences = False))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [20]:

from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn = vanilla_rnn, epochs = 30, batch_size = 50)
model.fit(X_train, y_train)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 50)                2600      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - 0s 1ms/step - loss: 0.6944 - accuracy: 0.4872
Epoch 2/30
1/1 [==============================] - 0s 1ms/step - loss: 0.6680 - accuracy: 0.6667
Epoch 3/30
1/1 [==============================] - 0s 1ms/step - loss: 0.6707 - accuracy: 0.5385
Epoch 4/30
1/1 [==============================] - 0s 1ms/step - loss: 0.6489 - accuracy: 0.641

In [21]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

print(accuracy_score(y_pred, y_test_))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
0.7


In [27]:
#a= ['The burger is good beef, cooked just right.     ']
a = ["It's too bad the food is so damn generic. "]
a = tokenizer.texts_to_sequences(a)
a=np.array(a)
a = pad_sequences(a, padding='post', maxlen=maxlen)

a = a.reshape((a.shape[0], a.shape[1], 1))
print(a.shape)

prediction = model.predict(np.array(a))
print(prediction) # 0 means negative and 1 means positive

(1, 100, 1)
[0]
